In [1]:
# 4번째 셀 결과에 결과 구조 있음

NameError: name 'accomm_info' is not defined

In [8]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re

from config import facilities_dict, region_dict

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
accomm_info = {}  # 숙소 정보 저장

page = 1
checkin = '2024-11-04'
checkout = '2024-11-08'
base_url = "https://www.yeogi.com/domestic-accommodations?"

for key, value in region_dict.items():
    url = f"{base_url}keyword={value}&autoKeyword=&checkIn={checkin}&checkout={checkout}&personal=2&freeForm=true&category=0"

    while page <= 2:
        current_url = f"{url}&page={page}"
        driver.get(current_url)
        time.sleep(5)
    
        # HTML 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        hotel_names = soup.find_all('h3', class_='gc-thumbnail-type-seller-card-title css-1asqkxl')
        type_elements = soup.find_all('ul', class_='css-19akvy6')
        price_elements = soup.find_all('span', class_='css-5r5920')
        ratings = soup.find_all('span', class_='css-9ml4lz')

        # href 속성에서 ID 추출
        accomm_ids = []
        links = soup.find_all('a', class_='gc-thumbnail-type-seller-card css-wels0m')
        for link in links:
            href = link['href']
            match = re.search(r'/domestic-accommodations/(\d+)', href)
        if match:
            accomm_id = match.group(1)
            accomm_ids.append(accomm_id)
        
        # 만약 현재 페이지에 데이터가 없으면 종료
        if not hotel_names:
            print("no hotel data exist")
            break
    
        # 숙소 정보 저장
        for accomm_id, hotel_name, type_element, price_element, rating in zip(accomm_ids, hotel_names, type_elements, price_elements, ratings):
            
            # 가격 정수로 변환
            price = int(price_element.text.replace(',', ''))
    
            # type 텍스트 구분 (블랙, 성급, 숙소 유형 등등..)
            types = [li.text for li in type_element.find_all('li')]
            black = None  # boolean으로 바꿔도 될 듯
            star = None
            hotel_type = None
            
            for t in types:
                if "성급" in t:
                    star = t
                elif "블랙" in t:
                    black = t
                else:
                    hotel_type = t
            
            accomm_info[accomm_id] = {
                "name": hotel_name.text,
                "address1": key,
                "black": black,
                "star": star,
                "type": hotel_type,
                "rating": float(rating.text),
                "price": price
            }
    
        # 다음 페이지로 이동
        page += 1
    page = 1
driver.quit()




In [132]:
# for name, info in accomm_info.items():
#     print(f"{name}: {info}")

수원역 J호텔-제이호텔: {'black': None, 'star': None, 'type': '모텔', 'rating': 8.9, 'price': 22500}
의정부 반월: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.4, 'price': 40500}
수원역 71번가-71st: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.1, 'price': 65500}
구리 호텔 인 카페: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.4, 'price': 75500}
의정부 STAGE79: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.6, 'price': 50500}
테이크 호텔 서울광명: {'black': '블랙', 'star': '4성급', 'type': '호텔', 'rating': 9.3, 'price': 60000}
성남 호텔 초콜릿-모란역점: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.6, 'price': 125000}
성남 HOTEL 월-모란역점: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.6, 'price': 27500}
인계동 No.25-수원인계점: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.2, 'price': 65500}
하남 호텔 트레비엔: {'black': None, 'star': None, 'type': '모텔', 'rating': 9.2, 'price': 80000}
르컬렉티브 시흥 웨이브파크: {'black': None, 'star': None, 'type': '호텔', 'rating': 9.3, 'price': 80500}
정왕 세븐헤르츠: {'black': 

In [18]:
import pandas as pd

df = pd.DataFrame(accomm_info)
df = pd.DataFrame.from_dict(accomm_info, orient='index')
df.index.name = 'Accommodations_ID'
df.reset_index(inplace=True)
df.rename(columns={
    'black': 'Black Mark',
    'star': 'Star',
    'type': 'Accommodation Type',
    'rating': 'Rating',
    'price': 'Price'
}, inplace=True)

In [141]:
# print(df.head())

       수원역 J호텔-제이호텔 의정부 반월 수원역 71번가-71st 구리 호텔 인 카페 의정부 STAGE79 테이크 호텔 서울광명  \
black          None   None          None       None        None          블랙   
star           None   None          None       None        None         4성급   
type             모텔     모텔            모텔         모텔          모텔          호텔   
rating          8.9    9.4           9.1        9.4         9.6         9.3   
price         22500  40500         65500      75500       50500       60000   

       성남 호텔 초콜릿-모란역점 성남 HOTEL 월-모란역점 인계동 No.25-수원인계점 하남 호텔 트레비엔  ...  세종 오즈  \
black            None            None            None       None  ...   None   
star             None            None            None       None  ...   None   
type               모텔              모텔              모텔         모텔  ...     모텔   
rating            9.6             9.6             9.2        9.2  ...    8.5   
price          125000           27500           65500      80000  ...  50000   

       세종 J호텔 춘천 세종호텔 세종 필사랑무인텔 광주역 세종호텔 세종 

In [20]:
df.to_csv('accommodation_data.csv', index=False)